# [Effective Approaches to Attention-based Neural Machine Translation](https://arxiv.org/abs/1508.04025)

TLDR; 著者たちはニューラル機械翻訳(NMT)向けの新たなattention機構、global attentionとlocal attentionを提案する(本論文提出時はNMTに関するattentionの記述は[Bahdanau et al. (2015)](https://arxiv.org/abs/1409.0473)しかなかった)。global attentionはNMT(sequence-to-sequence)の入力側全ての単語を用いて翻訳文を出力する。local attentionは入力側の一部の単語を用いて翻訳文を出力する。著者達はlocal attentionをWMT' 15英独翻訳タスクで評価しBLEUスコアで5.0の改善を達成した。

---

#### Key Points

* Goal; シンプルで有効なattention機構の導出
* attentionは一般に、異なるモダリティー間のalignmentを学習する枠組み
* global attentionはBahdanau et al. (2015)を真似ているが、よりシンプルなアーキテクチャーとなっている
* local attentionは[Xu et al., 2015](https://arxiv.org/abs/1502.03044)で提案されたイメージキャプション生成タスク向けのhardとsoft attentionをブレンドしたもの
  * soft attentionはglobal attentionのこと; 入力画像の全パッチを利用する
  * hard attentionは; 入力画像のパッチ一つを利用する、微分不可能という欠点を持つ
* 入力系列を$x_1, \cdots, x_n$、出力系列を$y_1, \cdots, y_m$、${\bf s}$をエンコーダーの与える入力系列の隠れ状態の層の系列としたとき、NMTは$x$を与えられた上で$y$を予測する: $\log p(y|x) = \sum_{j=1}^m \log p(y_j|y_{<j}, {\bf s}), \ p(y_j|y_{<j}, {\bf s}) = \textrm{softmax}(g({\bf h}_j)),\ {\bf h}_j = f({\bf h}_{j-1}, {\bf s})$
  * attentionを用いない場合、${\bf s}$は出力側の隠れ状態の層を初期化するために一度だけ用いる
  * attentionを用いる場合、翻訳処理全体を通して${\bf s}$を用いる
* Global Attention
  * alignmentの重みは(個人的に分かりにくかったので[tensorflow/nmt](https://github.com/tensorflow/nmt)の式を用いた): $a_{ts} = \frac{\exp(\textrm{score}({\bf h}_t, \overline{\bf h}_s))}{\sum_{s'=1}^S \exp(\textrm{score}({\bf h}_t, \overline{\bf h}_{s'}))}$
  * 文脈ベクトル${\bf c}_t$はこの重みで各入力層の隠れ層状態を加重平均したもので: ${\bf c}_t = \sum_s a_{ts} \overline{\bf h}_s$
  * attentionalベクトル$\overline{\bf h}_t$は: $\overline{\bf h}_t = \textrm{tanh}({\bf W}_{\bf c}[{\bf c}_t; {\bf h}_t])$
  * 最後に$y_t$は: $p(y_t|y_{<t}, x) = \textrm{softmax}({\bf W}_{\bf s} \tilde{\bf h}_t)$
  * $\textrm{score}$はcontent-based関数と呼ぶ、３つの変種:
    * dot: $\textrm{score}({\bf h}_t, \overline{\bf h}_s) = {\bf h}_t' \overline{\bf h}_s$
    * general: $\textrm{score}({\bf h}_t, \overline{\bf h}_s) = {\bf h}_t' {\bf W}_{\bf \alpha} \overline{\bf h}_s$
    * concat: $\textrm{score}({\bf h}_t, \overline{\bf h}_s) = {\bf v}_{\alpha}' \textrm{tanh}({\bf W}_{\bf \alpha} [{\bf h}_t'; \overline{\bf h}_s])$
  * もう一つの変種、location-based: ${\bf \alpha}_t = \textrm{softmax}({\bf W}_{\bf \alpha} {\bf h}_t)$
  * Bahdanau et al. (2015)との違い: よりシンプルによりgeneralに
    * 本workではエンコーダーとデコーダーのLSTM層の最上位層の隠れ状態を使ってる
    * 本workでは計算パスを簡素化してる: ${\bf h}_t \to {\bf \alpha}_t \to {\bf c}_t \to \tilde{\bf h}_t$
    * concat以外のalignmentも試した
* Local Attention
  * 出力側の単語毎に入力側の単語の部分集合を用いるというアイデア
    1. まず時刻$t$の出力側の各単語のためにalignedされた場所$p_t$を生成する
    2. 文脈ベクトル${\bf c}_t$は、ウィンドウ$[p_t - D, p_t + D]$の中の入力側の隠れ状態の加重平均
* Input-feeding Approach: attentionalベクトル$\tilde{\bf h}_t$を次のタイムステップの入力と連結して次のタイムステップの予測に用いる
* WMTの英独間の双方向の翻訳でモデルを評価
  * 積層LSTM、4層、1000次元
* attentionalモデルはnon-attentionalモデルより長い文章を扱うことにより効率的である: 文章が長くなっても質はでグレードしない
* location-based関数は良いalignmentを学習しない
* global(location)モデルは未知語の置換をしたときに少しの改善のみ得られた
* content-based関数ではconcatは良いパフォーマンスを呈さなかった
* dotはglobal attentionで、generalはlocal attentionで良かった
* local-pが最も良かった
* attentionalモデルは“Miranda Kerr”や“Roger Dow”などの名前を正しく翻訳している、non-attentionalモデルは言語モデルの観点から意味のある名前を生成している一方けど、入力側の情報を上手く用いておらず間違った名前を出力してる

#### Thoughts

* 最近はglobal attentionの方しか見かけない気がするが、どういう経緯があったのだろう？

#### 参考

* Luong, Minh-Thang, Hieu Pham, and Christopher D. Manning. "Effective approaches to attention-based neural machine translation." arXiv preprint arXiv:1508.04025 (2015).
* Bahdanau, Dzmitry, Kyunghyun Cho, and Yoshua Bengio. "Neural machine translation by jointly learning to align and translate." arXiv preprint arXiv:1409.0473 (2014).
* Xu, Kelvin, et al. "Show, attend and tell: Neural image caption generation with visual attention." International Conference on Machine Learning. 2015.
* tensorflow/nmt
